In [1]:
import pandas as pd
import gc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.chdir('../')
from model import *
os.chdir('../../data/')

In [2]:
transactions = pd.read_pickle('../data/compressed_data/transactions_train.pkl')
customers = pd.read_pickle('../data/compressed_data/customers.pkl')
articles = pd.read_pickle('../data/compressed_data/articles.pkl')

transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

print('First week num: ', transactions.week.min(), '\nLast week num: ', transactions.week.max(), '\n')

First week num:  0 
Last week num:  104 



In [3]:
# Test week is week after last week in train data
test_week = transactions.week.max() + 1

# Filter transactions to last 10 weeks (most recent data)
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [4]:
transactions.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'week'],
      dtype='object')

In [5]:
# Load radek's candidates
candidates_last_purchase = pd.read_csv('../data/candidates/radek_last_purchase.csv')
candidates_bestsellers = pd.read_csv('../data/candidates/radek_bestsellers.csv')
bestsellers_previous_week = pd.read_csv('../data/candidates/radek_bestsellers_previous_week.csv')

# Load my candidates

In [6]:
seasonal_bestsellers = pd.read_csv('../data/candidates_200_ranks/seasonal_candidates_2019.csv')

## Submission Loop -- check best k for kaggle

In [7]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank',
    'seasonal_bestseller_rank',
]

model_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'dart',
    'n_estimators': 1,
    'importance_type': 'gain'
}

In [8]:
# Get bestsellers from previous week
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week['week'] == bestsellers_previous_week['week'].max()]['article_id'].tolist()
 
for k in range(20, 201, 10):
    print('k = ', k)

    seasonal_candidates_filtered = seasonal_bestsellers[seasonal_bestsellers.article_id.isin(transactions.article_id.unique())].head(k)

    seasonal_candidates = transactions[['customer_id', 'week']].drop_duplicates()
    test_candidates = customers[customers.customer_id.isin(transactions.customer_id.unique())][['customer_id']].drop_duplicates()
    test_candidates['week'] = test_week
    seasonal_candidates = pd.concat([seasonal_candidates, test_candidates], ignore_index=True)

    seasonal_candidates['key'] = 1
    seasonal_candidates_filtered['key'] = 1

    seasonal_candidates = seasonal_candidates.merge(seasonal_candidates_filtered, on='key').drop(columns='key')
    seasonal_candidates['t_dat'] = '2020-07-15'
    seasonal_candidates['price'] = 0
    seasonal_candidates['sales_channel_id'] = 2
    seasonal_candidates.drop(columns=['seasonal_bestseller_rank', 'year'], inplace=True)


    # Prepare data for model
    train_X, train_y, test_X, test, train_baskets = prepare_data(
        transactions,
        bestsellers_previous_week,
        candidates=[candidates_last_purchase, candidates_bestsellers, seasonal_candidates], 
        features=[customers, articles, seasonal_bestsellers[['article_id', 'seasonal_bestseller_rank']]], 
        cols_to_use=columns_to_use
        )
    
    gc.collect()
    
    # Train model
    ranker = train_model(
        train_X, 
        train_y, 
        train_baskets, 
        model_params, 
        columns_to_use, 
        show_importance=10
    )

    gc.collect()

    # Make submission
    make_submission(customers, test, test_X, ranker, bestsellers_last_week, f'submission_seasonal_{k}')

    del seasonal_candidates, test_candidates, seasonal_candidates_filtered, train_X, train_y, test_X, test, train_baskets, ranker
    gc.collect()


k =  20
Percentage of real transactions:  0.061829279882676676
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.430650 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1184
[LightGBM] [Info] Number of data points in the train set: 23463051, number of used features: 19
Feature importance:
seasonal_bestseller_rank 0.5720376883311716
bestseller_rank 0.4272203347278425
age 0.0002152678015759515
garment_group_no 0.00013095893320647876
article_id 0.0001212617767094929
section_no 9.965273324791193e-05
department_no 8.115657964707224e-05
perceived_colour_master_id 4.078699575167992e-05
perceived_colour_value_id 3.7758158809847746e-05
index_group_no 1.5133962037420627e-05
Starting submission process...
Calculating predictions...
Done.
Creating submi

100%|██████████| 58.0M/58.0M [00:26<00:00, 2.27MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  30
Percentage of real transactions:  0.04821502177484468
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.648654 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1168
[LightGBM] [Info] Number of data points in the train set: 30100035, number of used features: 19
Feature importance:
seasonal_bestseller_rank 0.6717934027516975
bestseller_rank 0.32725238051124683
age 0.00028558696564934716
garment_group_no 0.00016245883065897716
section_no 0.00014365970399203028
article_id 0.0001155961450922189
department_no 0.0001059922758259242
perceived_colour_master_id 5.5369998108101734e-05
perceived_colour_value_id 4.372388306081759e-05
colour

100%|██████████| 58.1M/58.1M [00:27<00:00, 2.23MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  40
Percentage of real transactions:  0.03920390825982366
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.796202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1167
[LightGBM] [Info] Number of data points in the train set: 36800401, number of used features: 19
Feature importance:
seasonal_bestseller_rank 0.7374803161338307
bestseller_rank 0.2613269050650377
age 0.0002824484684367767
article_id 0.0002209856933250561
garment_group_no 0.00019414791287649915
section_no 0.00015580035500085311
department_no 0.00013197095291825296
perceived_colour_master_id 6.8497682382352e-05
colour_group_code 5.216607979365776e-05
perceived_colour_v

...

Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  190
Percentage of real transactions:  0.01220610208689714
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.857371 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1179
[LightGBM] [Info] Number of data points in the train set: 116243133, number of used features: 19
Feature importance:
seasonal_bestseller_rank 0.9312218828985023
bestseller_rank 0.06551944730042172
article_id 0.0006387492049801384
garment_group_no 0.0005110400146892217
perceived_colour_master_id 0.0005087660411255784
age 0.0004987234504363261
section_no 0.0004444942466204021
product_type_no 0.000332931268098815
department_no 0.00013481493411381177
colour_group_code 

100%|██████████| 57.9M/57.9M [00:28<00:00, 2.11MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  200
Percentage of real transactions:  0.01220610208689714
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.545749 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1179
[LightGBM] [Info] Number of data points in the train set: 116243133, number of used features: 19
Feature importance:
seasonal_bestseller_rank 0.9312218828985023
bestseller_rank 0.06551944730042172
article_id 0.0006387492049801384
garment_group_no 0.0005110400146892217
perceived_colour_master_id 0.0005087660411255784
age 0.0004987234504363261
section_no 0.0004444942466204021
product_type_no 0.000332931268098815
department_no 0.00013481493411381177
colour_group_code 

100%|██████████| 57.9M/57.9M [00:29<00:00, 2.07MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.


In [10]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 21

fileName                        date                 description              status    publicScore  privateScore  
------------------------------  -------------------  -----------------------  --------  -----------  ------------  
submission_seasonal_200.csv.gz  2023-12-14 14:21:39  submission_seasonal_200  complete  0.01973      0.01958       
submission_seasonal_190.csv.gz  2023-12-14 14:07:21  submission_seasonal_190  complete  0.01973      0.01958       
submission_seasonal_180.csv.gz  2023-12-14 13:52:22  submission_seasonal_180  complete  0.01973      0.01958       
submission_seasonal_170.csv.gz  2023-12-14 13:36:43  submission_seasonal_170  complete  0.01973      0.01958       
submission_seasonal_160.csv.gz  2023-12-14 13:20:40  submission_seasonal_160  complete  0.01973      0.01958       
submission_seasonal_150.csv.gz  2023-12-14 13:05:26  submission_seasonal_150  complete  0.01963      0.01951       
submission_seasonal_140.csv.gz  2023-12-14 12:50:51  submission_seasonal